## Playing with data frames in a DataBricks cluster
### This notebook needs a databricks cluster to execute. Cannot run on local Jupyter notebook
#### This tutorial is part of "A Gentle Introduction to Spark" by Databricks
The pdf url is:
https://pages.databricks.com/rs/094-YMS-629/images/A-Gentle-Introduction-to-Apache-Spark.pdf?mkt_tok=eyJpIjoiWXpsbVlqVXpaV1psWlRsaCIsInQiOiJ1VWNPYnQxZjUxR3Y0a2o2ZzFZQ0ZWVjU0M1pKOW9MNzFvR3Q3Ymx1ZlY1XC9qbjVyUzQzRVk1ek5kWGF5a3JXTXNPd3Y1SENEM1k1eUpqVFRIYkkrb0pvUGVxbTlOM3NNQWEwemo2K29lZVpsRnVzdW1xbjRBTm5KRHZuUFJuSFgifQ%3D%3D

Data Location:
https://github.com/databricks/Spark-The-Definitive-Guide/tree/master/data/flight-data/csv


In [18]:
import yaml

with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

container= cfg['container']
storage_account= cfg['storage_account']
storage_key= cfg['storage_key']
secret= cfg['secret']

In [ ]:
mytable = spark.table("summary2015")
display(mytable.select("*"))

In [ ]:
dbutils.fs.mount(
  source = "wasbs://" + container +"@" + storage_account + ".blob.core.windows.net/",
  mount_point = "/mnt/flight-data",
  extra_configs = {""+ storage_key +"": ""+ secret +""})

In [5]:

flightData2015=spark\
  .read\
  .option("inferSchema","true")\
  .option("header","true")\
  .csv("/mnt/flight-data/2015-summary.csv")

In [6]:
flightData2015.take(3)

In [7]:
flightData2015.sort("count").explain()

In [8]:
spark.conf.set("spark.sql.shuffle.partitions","5")
flightData2015.sort("count").take(2)

In [9]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [10]:
sqlWay=spark.sql("""
              SELECT DEST_COUNTRY_NAME, count(1) 
              FROM flight_data_2015
              GROUP BY DEST_COUNTRY_NAME
              """)

dataFrameWay=flightData2015\
            .groupBy("DEST_COUNTRY_NAME")\
            .count()
#display(sqlWay)
#display(dataFrameWay)

sqlWay.explain()
dataFrameWay.explain()